Dervied from Example at:
https://mygeoblog.com/2019/08/21/google-earth-engine-to-numpy/

In [1]:
import ee
import numpy as np
#from osgeo import gdal
#from osgeo import osr
import time
 
# init the ee object
ee.Initialize()
 
 
 


In [2]:
# Define the roi
#brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
area = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
#area = ee.Geometry.Rectangle(-60.25, -2.70, -60.00, -2.50)
#area = ee.Geometry.Polygon([[105.4084512289977,12.960956032145036],\
#                            [105.46544280614614,12.960956032145036],\
#                            [105.46544280614614,13.006454200439705],\
#                            [105.4084512289977,13.006454200439705],\
#                            [105.4084512289977,12.960956032145036]])
 
# define the image
collection = ee.ImageCollection("COPERNICUS/S2").filterBounds(area)\
                                      .filterDate("2019-01-01","2019-01-31")\
                                      .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50)\
                                      .select(['B8', 'B5'])
 
print(" number of image: ",collection.size().getInfo())
#print(collection)

 number of image:  12


In [3]:
# perform any calculation on the image collection here
def addNDRE(img):
    ndre = ee.Image(img.normalizedDifference(['B8', 'B5'])).rename(["ndre"])
    return ndre
 
# export the latitude, longitude and array
def LatLonImg(img):
    img = img.addBands(ee.Image.pixelLonLat())
 
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=area,\
                                        maxPixels=1e50,\
                                        scale=10);
 
    data = np.array((ee.Array(img.get("result")).getInfo()))
    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    return lats, lons, data
 
# covert the lat, lon and array into an image
def toImage(lats,lons,data):
 
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
 
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)
    nrows = len(uniqueLats)
 
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0]
    xs = uniqueLons[1] - uniqueLons[0]
 
    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32) #-9999
 
    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    return arr


In [4]:
# map over the image collection
ndreCollection  = collection.map(addNDRE)
 
# get the median
ndreMedian = ee.Image(ndreCollection.median()).rename(['results'])
 
## get the lon, lat and result as 1d array
#lat, lon, data = LatLonImg(ndreMedian)
 
# 1d to 2d array
#image  = toImage(lat,lon,data)
 
# in case you want to plot the image
#import matplotlib.pyplot as plt
#plt.imshow(image)
#plt.show()

In [74]:
# Jitu Kumar: Adding functions to calculate regional statistics using ee.Reducers routines

# Mean
def region_mean_Image(img):
    # mean over all pixels in the region for the image
    return img.set('mean', img.reduceRegion(ee.Reducer.mean(), geometry=area, scale=20)) 

def region_mean_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_mean_Image)
    stats_list = stats.aggregate_array('mean')
    return np.array(stats_list.getInfo())

# Median
def region_median_Image(img):
    # mean over all pixels in the region for the image
    return img.set('median', img.reduceRegion(ee.Reducer.median(), geometry=area, scale=20)) 

def region_median_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_median_Image)
    stats_list = stats.aggregate_array('median')
    return np.array(stats_list.getInfo())

# Stddev
def region_stddev_Image(img):
    # stddev over all pixels in the region for the image
    return img.set('stddev', img.reduceRegion(ee.Reducer.stdDev(), geometry=area, scale=20)) 

def region_stddev_ImageCollection(ic):
    # stddev over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_stddev_Image)
    stats_list = stats.aggregate_array('stddev')
    return np.array(stats_list.getInfo())

# Percentiles
def region_percentiles_Image(img):
    # 5,10,25,75,90,95 percentiles over all pixels in the region for the image
    return img.set('percentiles', img.reduceRegion(ee.Reducer.percentile([5,10,25,75,90,95]), geometry=area, scale=20)) #) mean, median, stdDev, percentiles

def region_percentiles_ImageCollection(ic):
    # percentiles over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_percentiles_Image)
    stats_list = stats.aggregate_array('percentiles')
    return np.array(stats_list.getInfo())
    

In [75]:
mean_list = region_mean_ImageCollection(ndreCollection)
median_list = region_median_ImageCollection(ndreCollection)
stddev_list = region_stddev_ImageCollection(ndreCollection)
percentiles_list = region_percentiles_ImageCollection(ndreCollection)

In [76]:
print(mean_list)

[{'ndre': 0.2444862373196642} {'ndre': 0.24265197156912838}
 {'ndre': 0.21672902684661105} {'ndre': 0.2218715198468275}
 {'ndre': 0.22110884993185262} {'ndre': 0.32352893896930734}
 {'ndre': 0.423432421350117} {'ndre': 0.37027115547613937}
 {'ndre': 0.43552997784775926} {'ndre': 0.39778245154816105}
 {'ndre': 0.4381995630705758} {'ndre': 0.21119927409490866}]


In [77]:
print(percentiles_list)

[{'ndre_p10': 0.04294852977891597, 'ndre_p25': 0.13678747218617207, 'ndre_p5': 0.019594792023632936, 'ndre_p75': 0.3319874416011698, 'ndre_p90': 0.5038777219840984, 'ndre_p95': 0.5664725630823043}
 {'ndre_p10': 0.01949448240887162, 'ndre_p25': 0.08202751613910964, 'ndre_p5': 0.004056480356633859, 'ndre_p75': 0.39454535092601933, 'ndre_p90': 0.5352249603947525, 'ndre_p95': 0.5741810138862006}
 {'ndre_p10': 0.1269675936456005, 'ndre_p25': 0.15821982583840621, 'ndre_p5': 0.10745101168078015, 'ndre_p75': 0.25584925889381116, 'ndre_p90': 0.3300633197226467, 'ndre_p95': 0.3964963929193256}
 {'ndre_p10': 0.0195144999364196, 'ndre_p25': 0.08982455344640713, 'ndre_p5': 0.0040872436220549604, 'ndre_p75': 0.31639518316996507, 'ndre_p90': 0.4804261067965295, 'ndre_p95': 0.5586554540028115}
 {'ndre_p10': 0.011781982326922699, 'ndre_p25': 0.042891425152814695, 'ndre_p5': -0.003626410028946759, 'ndre_p75': 0.3945010943825622, 'ndre_p90': 0.5509036275235385, 'ndre_p95': 0.574207353391762}
 {'ndre_p10'